In [68]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sb
import torch
import torch.nn as nn
import numpy as np
import plotly.express as px
import plotly.graph_objects as go  
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_absolute_error, mean_squared_error, classification_report
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [2]:
data = yf.download('AAPL', start='2018-01-01', end='2025-06-01')

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [3]:
print(data.head())

Price           Close       High        Low       Open     Volume
Ticker           AAPL       AAPL       AAPL       AAPL       AAPL
Date                                                             
2018-01-02  40.426815  40.436204  39.722760  39.933979  102223600
2018-01-03  40.419788  40.964259  40.356426  40.490195  118071600
2018-01-04  40.607525  40.710787  40.384575  40.492528   89738400
2018-01-05  41.069866  41.156698  40.612231  40.703758   94640000
2018-01-08  40.917324  41.213026  40.818753  40.917324   82271200


In [4]:
df = data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1863 entries, 2018-01-02 to 2025-05-30
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   (Open, AAPL)    1863 non-null   float64
 1   (High, AAPL)    1863 non-null   float64
 2   (Low, AAPL)     1863 non-null   float64
 3   (Close, AAPL)   1863 non-null   float64
 4   (Volume, AAPL)  1863 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 87.3 KB


In [6]:
df.isnull().sum()

Price   Ticker
Open    AAPL      0
High    AAPL      0
Low     AAPL      0
Close   AAPL      0
Volume  AAPL      0
dtype: int64

In [7]:
df.describe()

Price,Open,High,Low,Close,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
count,1863.000000,1863.000000,1863.000000,1863.000000,1.863000e+03
mean,127.300563,128.735444,125.980373,127.432247,9.787064e+07
std,61.763485,62.392101,61.183128,61.837595,5.484645e+07
min,34.297229,34.711713,33.825578,33.870838,2.323470e+07
25%,59.622267,60.242862,58.158031,59.401499,6.003700e+07
50%,138.394745,140.073786,136.009644,138.299835,8.449680e+07
75%,172.565170,174.630462,171.412979,172.540756,1.188404e+08
max,257.568678,259.474086,257.010028,258.396667,4.265100e+08


In [8]:
df.shape

(1863, 5)

In [9]:
def computeRSI(data, periods=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=periods).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

In [10]:
df['RSI'] = computeRSI(df['Close'])

In [11]:
df = df.dropna()

# Data Preprocessing

In [12]:
scaler = MinMaxScaler()
dfScaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
closeScaler = MinMaxScaler()
closeScaler.fit(df[['Close']])

MinMaxScaler()

# create sequences e.g., 60-day windows

In [13]:
def createSequences(data, seqLength=60):
    X, y = [], []
    for i in range(len(data) - seqLength):
        X.append(data[i:i + seqLength, [0, 1, 2, 4, 5]])
        #X.append(data[i:i + seqLength]) # Shape: [seqLenght, 5]
        y.append(data[i + seqLength, 3]) # Index 3 is Close
    
    return np.array(X), np.array(y)

In [14]:
#features = dfScaled[['Open', 'High', 'Low', 'Close', 'Volume']].values
features = dfScaled[['Open', 'High', 'Low', 'Close', 'Volume', 'RSI']].values

X, y = createSequences(features)
print(f"X shape: {X.shape}, y shape: {y.shape}")

X shape: (1790, 60, 5), y shape: (1790,)


# Create train test splits

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to tensors

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [17]:
X_trainTensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_testTensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_trainTensor = torch.tensor(y_train, dtype=torch.float32).to(device)
y_testTensor = torch.tensor(y_test, dtype=torch.float32).to(device)
print("X_trainTensor shape:", X_trainTensor.shape)  # Debug


X_trainTensor shape: torch.Size([1432, 60, 5])


# Create Model

## Using LSTM (Long Short-Term Memory) 
Reasons for using LSTM is thats its a type of recurrent neural network (RNN). That is effective at handling time series data.

In [ ]:
class StockLSTM(nn.Module):
    def __init__(self, inputSize=5, hiddenUnits=100, dropout1=0.2, dropout2=0.3):
        super(StockLSTM, self).__init__()
        # First LSTM layer
        self.lstm1 = nn.LSTM(inputSize, hiddenUnits, num_layers=1, batch_first=True, dropout=dropout1)
        # Second LSTM layer
        self.lstm2 = nn.LSTM(hiddenUnits, hiddenUnits, num_layers=1, batch_first=True, dropout=dropout2)
        # Third LSTM layer 
        self.lstm3 = nn.LSTM(hiddenUnits, hiddenUnits, num_layers=1, batch_first=True) # No dropout for last

        # Fully connected layer
        self.fc = nn.Linear(hiddenUnits, 1) 

    def forward(self, x):
        # Pass through first LSTM
        # out1 will have shape (batch_size, sequence_length, hiddenUnits)
        out1, _ = self.lstm1(x)

        # Pass the output of lstm1 as input to lstm2
        out2, _ = self.lstm2(out1)

        # Pass the output of lstm2 as input to lstm3
        out3, _ = self.lstm3(out2)

        # Take the output of the *last* time step from the *last* LSTM layer (lstm3)
        final_output = out3[:, -1, :] # Shape: (batch_size, hidden_units)

        # Pass to fully connected layer
        prediction = self.fc(final_output)
        return prediction

# Training Loop Function

In [19]:
def train(model, X_train, y_train, X_test, y_test, loss_fn, epochs, optimizer, device):
    model.to(device)
    trainDataset = TensorDataset(X_train, y_train)
    trainLoader = DataLoader(trainDataset, batch_size=32, shuffle=True)

    trainLosses = []
    for epoch in tqdm(range(epochs)):
        model.train()
        epochLoss = 0
        for X_batch, y_batch in trainLoader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch.unsqueeze(1)) # Make sure y_batch shape matches
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epochLoss += loss.item()
        
        trainLosses.append(epochLoss / len(trainLoader))
        if epoch % 10 == 0:
            print(f'Epoch: {epoch}, Train Loss: {trainLosses[-1]:.4f}')

    return trainLosses

# Train Model

In [65]:
model1 = StockLSTM(inputSize=5, hiddenUnits=100)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model1.parameters(), lr=0.001)
resultsTrain = train(
                model1, 
                X_trainTensor,
                y_trainTensor,
                X_testTensor, 
                y_testTensor, 
                loss_fn=loss_fn, 
                epochs=100, 
                optimizer=optimizer, 
                device=device
            )

c:\Users\Edward\Desktop\Projects\ML\Stock Price Forcast YFinance\venv\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning:

dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1

c:\Users\Edward\Desktop\Projects\ML\Stock Price Forcast YFinance\venv\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning:

dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1

  2%|▏         | 2/100 [00:00<00:24,  3.95it/s]

Epoch: 0, Train Loss: 0.0495


 12%|█▏        | 12/100 [00:01<00:12,  6.86it/s]

Epoch: 10, Train Loss: 0.0005


 22%|██▏       | 22/100 [00:03<00:11,  6.98it/s]

...

 82%|████████▏ | 82/100 [00:11<00:02,  7.31it/s]

Epoch: 80, Train Loss: 0.0002


 92%|█████████▏| 92/100 [00:13<00:01,  7.02it/s]

Epoch: 90, Train Loss: 0.0003


100%|██████████| 100/100 [00:14<00:00,  6.99it/s]


# Evaluate Model

In [63]:
def evaluateModel(model, X_test, y_test, scaler, device):
    model.eval()
    with torch.no_grad():
        preds = model(X_test).cpu().numpy()
        y_testNp = y_test.cpu().numpy()
    
    preds = scaler.inverse_transform(preds)
    y_testNp = scaler.inverse_transform(y_testNp.reshape(-1, 1))
    mae = mean_absolute_error(y_testNp, preds)
    rmse = np.sqrt(mean_squared_error(y_testNp, preds))
    maePercentage = (mae / np.mean(y_testNp)) * 100
    print(f'MAE: {mae:.4f}, RMSE: {rmse:.4f}, MAE %: {maePercentage:.2f}%')

    return preds, y_testNp, mae, rmse

In [66]:
predictions, y_testNp, mae, rmse = evaluateModel(model1, X_testTensor, y_testTensor, closeScaler, device)

MAE: 1.8958, RMSE: 2.5867, MAE %: 1.44%


# Plot Predictions

In [42]:
def plotPredictions(actual, predicted, title='AAPL Close Price Forecasting', savePath='plot'):
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=actual.flatten(), name='Actual Close Prices', line=dict(color='blue')))
    fig.add_trace(go.Scatter(y=predicted.flatten(), name='Predicted Close Prices', line=dict(color='orange')))
    fig.update_layout(
        title=title,
        xaxis_title='Time',
        yaxis_title='Close Price (USD)',
        template='plotly_dark',
        hovermode='x unified'
    )

    fig.show()

In [67]:
plotPredictions(y_testNp, predictions)

In [70]:
def plotErrorOverTime(actual, predicted, title='AAPL Prediction Error Over Time'):
    indices = np.arange(len(actual))
    errors = np.abs(actual.flatten() - predicted.flatten())
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=indices, y=errors, name='Absolute Error', line=dict(color='red')))
    fig.update_layout(
        title=title,
        xaxis_title='Date',
        yaxis_title='Absolute Error (USD)',
        template='plotly_dark',
        hovermode='x unified',
        xaxis=dict(tickformat='%Y-%m-%d', tickangle=45)
    )
    fig.show()

In [71]:
plotErrorOverTime(y_testNp, predictions)

In [72]:
def plotErrorCDF(actual, predicted, title='AAPL Predicted Error CDF'):
    errors = np.abs(actual.flatten() - predicted.flatten())
    sortedErrors = np.sort(errors)
    cdf = np.arange(1, len(sortedErrors) + 1) / len(sortedErrors)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=sortedErrors, y=cdf, name='CDF', line=dict(color='green')))
    fig.update_layout(
        title=title,
        xaxis_title='Absolute Error (USD)',
        yaxis_title='Cumulative Probability',
        template='plotly_dark',
        hovermode='x unified'
    )
    fig.show()

In [73]:
plotErrorCDF(y_testNp, predictions)